# Model $\Lambda_c \to p \pi K$

<!-- cspell:ignore onlyls Xlineshape -->

In [ ]:
using Markdown
using InteractiveUtils

In [ ]:
begin
  cd(joinpath(@__DIR__, ".."))
  using Pkg
  Pkg.activate(".")
  Pkg.instantiate()
  #
  using JSON
  using Plots
  using RecipesBase
  #
  using LinearAlgebra
  using Parameters
  #
  using ThreeBodyDecay
  using ThreeBodyDecay.PartialWaveFunctions
end

In [ ]:
theme(:wong2, frame=:box, grid=false, minorticks=true,
  guidefontvalign=:top, guidefonthalign=:right,
  xlim=(:auto, :auto), ylim=(:auto, :auto),
  lw=1, lab="", colorbar=false)

In [ ]:
begin
  const mK = 0.493
  const mπ = 0.140
  const mp = 0.938
  const mΣ = 1.18925
  const mΛc = 2.28
end

const ms = ThreeBodyMasses(m1=mp, m2=mπ, m3=mK, m0=mΛc)
const tbs = ThreeBodySystem(ms, ThreeBodySpins(1, 0, 0; two_h0=1))
const parities = ['+', '-', '-', '±']

In [ ]:
function readjson(path)
  f = read(path, String)
  return JSON.parse(f)
end

## Lineshapes

In [ ]:
breakup(m², m1², m2²) = sqrt(Kallen(m², m1², m2²))

### `BreitWignerMinL`

In [ ]:
begin
  @with_kw struct BreitWignerMinL{T}
    pars::T
    l::Int
    minL::Int
    #
    name::String
    #
    m1::Float64
    m2::Float64
    mk::Float64
    m0::Float64
  end
  BreitWignerMinL(pars::T; kw...) where {T} = BreitWignerMinL(; pars, kw...)
  function (BW::BreitWignerMinL)(s, σ)
    m, Γ₀ = BW.pars
    @unpack l, minL = BW
    @unpack m1, m2, mk, m0 = BW
    p, p0 = breakup(σ, m1^2, m2^2), breakup(m^2, m1^2, m2^2)
    q, q0 = breakup(s, σ, mk^2), breakup(s, m^2, mk^2)
    Γ = Γ₀ * (p / p0)^(2l + 1) * m / sqrt(σ)  # should also have F
    1 / (m^2 - σ - 1im * m * Γ) * (p / p0)^l * (q / q0)^minL # should also have F
  end
end

### `BuggBreitWignerMinL`

In [ ]:
begin
  @with_kw struct BuggBreitWignerMinL{T}
    pars::T
    l::Int
    minL::Int
    #
    name::String
    #
    m1::Float64
    m2::Float64
    mk::Float64
    m0::Float64
  end
  BuggBreitWignerMinL(pars::T; kw...) where {
    T<:NamedTuple{X,Tuple{Float64,Float64}}} where {X} =
    BuggBreitWignerMinL(; pars=merge(pars, (γ=1.1,)), kw...)
  #
  function (BW::BuggBreitWignerMinL)(s, σ)
    σA = mK^2 - mπ^2 / 2
    m, Γ₀, γ = BW.pars
    mΓ = (σ - σA) / (m^2 - σA) * Γ₀^2 * exp(-γ * σ)
    1 / (m^2 - σ - 1im * mΓ)
  end
end

### `Flatte1405`

In [ ]:
begin
  @with_kw struct Flatte1405{T}
    pars::T
    l::Int
    minL::Int
    #
    name::String
    #
    m1::Float64
    m2::Float64
    mk::Float64
    m0::Float64
  end
  #
  Flatte1405(pars::T; kw...) where {T} = Flatte1405(; pars, kw...)
  function (BW::Flatte1405)(s, σ)
    m, Γ₀ = BW.pars
    @unpack m1, m2 = BW
    p, p0 = breakup(σ, m1^2, m2^2), breakup(m^2, m1^2, m2^2)
    p′, p0′ = breakup(σ, mπ^2, mΣ^2), breakup(m^2, mp^2, mΣ^2)
    Γ1 = Γ₀ * (p / p0) * m / sqrt(σ)  # should also have F
    Γ2 = Γ₀ * (p′ / p0′) * m / sqrt(σ)  # should also have F
    Γ = Γ1 + Γ2
    1 / (m^2 - σ - 1im * m * Γ)
  end
end

## Parsing the input


In [ ]:
function ifhyphenaverage(s::String)
  factor = findfirst('-', s) === nothing ? 1 : 2
  eval(Meta.parse(replace(s, '-' => '+'))) / factor
end

In [ ]:
function buildchain(key, dict)
  @unpack mass, width, lineshape = dict
  #
  k = Dict('K' => 1, 'D' => 3, 'L' => 2)[first(key)]
  #
  parity = dict["jp"][end]
  two_j = Int(2 * eval(Meta.parse(dict["jp"][1:end-2])))
  #
  massval, widthval = ifhyphenaverage.((mass, width)) ./ 1e3
  #
  LS = vcat(
    [possible_ls(
      jp(two_j // 2, parity),
      jp(tbs.two_js[k] // 2, parities[k]);
      jp=jp(tbs.two_js[0] // 2, P0)) for P0 in ['+', '-']]...)
  #
  i, j = ij_from_k(k)
  ls = possible_ls(
    jp(tbs.two_js[i] // 2, parities[i]),
    jp(tbs.two_js[j] // 2, parities[j]);
    jp=jp(two_j // 2, parity))
  length(ls) != 1 && error("weird")
  onlyls = first(ls)
  minLS = first(sort(vcat(LS...); by=x -> x[1]))
  #
  code = quote
    decay_chain(
      k=$k,
      Xlineshape=$(Symbol(lineshape))(
        (; m=$massval, Γ=$widthval);
        name=$key,
        l=$(onlyls[1]),
        minL=$(minLS[1]),
        m1=$(ms[i]), m2=$(ms[j]), mk=$(ms[k]), m0=$(ms[0])),
      two_LS=$(minLS .|> x2),
      two_ls=$(onlyls .|> x2),
      two_s=$two_j, tbs=$tbs,
    )
  end
  eval(code)
  # code
end

In [ ]:
isobarsinput = readjson(joinpath("..", "data", "isobars.json"))["isobars"];

In [ ]:
ds = [buildchain(keys, dict) for (keys, dict) in isobarsinput]

In [ ]:
let
  plot(layout=grid(1, 3), size=(1000, 240))
  for d in ds
    plot!(sp=d.k, x -> abs2(d.Xlineshape(ms[0]^2, x)), lims(d.k, ms)...,
      lab=d.Xlineshape.name)
  end
  plot!()
end